# Introduction | Using Nimble for Live Weather Tracking Information

This notebook is intended to show how to use [Nimble's remote MCP server](https://docs.nimbleway.com/ai-agents/mcp-server) with Databricks to extract live weather alerting data

# 🌦️ Introduction | Using Nimble for Live Weather Tracking Information

This notebook demonstrates how to integrate **Nimble's Model Context Protocol (MCP) server** with **Databricks** to enable **live weather tracking and alerting** workflows.

With the growing impact of extreme weather conditions, real-time awareness is critical for applications in:
- Emergency response
- Transportation and logistics
- Supply chain risk management
- Insurance and underwriting
- Public safety alert systems

## 🔧 What is the MCP Server?

The **MCP server** allows Databricks agents to query external data sources in real-time with context awareness. When connected to a remote weather endpoint (such as NOAA, OpenWeatherMap, or trusted regional sources), agents can extract and interpret live data like:
- Severe weather warnings (floods, tornadoes, storms)
- Regional temperature anomalies
- Wind and precipitation alerts
- Real-time satellite-based observations

## ✅ Key Use Cases for Weather Alerting with MCP

1. **Localized Alerting Dashboards**
   - Use MCP queries to populate interactive dashboards for different regions.
   - Example: “Show all active severe weather alerts for California.”

2. **Proactive Notifications via Agent**
   - Create a Databricks agent that polls MCP-connected weather feeds and sends Slack or SMS alerts to users in affected ZIP codes.
   - Example: “If wind speed > 60 mph near any company facility, trigger alert.”

3. **ML-Driven Risk Estimation**
   - Use MCP weather context in model training or scoring for insurance, logistics, or agriculture.
   - Example: “Fetch past 30 days of rainfall data before estimating crop yield risks.”

## 🔐 Ensuring Trusted Weather Data

To maintain accuracy and integrity of weather alerts, always **limit MCP requests to reputable data providers**:

| Source | Use Case | Endpoint |
|--------|----------|----------|
| NOAA (National Weather Service) | U.S. weather warnings & alerts | `https://api.weather.gov/alerts` |
| OpenWeatherMap | Global real-time weather data | `https://api.openweathermap.org/data/2.5/weather` |
| Meteomatics | High-resolution global weather | `https://api.meteomatics.com/` |
| Environment Canada | Canadian severe weather alerts | `https://dd.weather.gc.ca/alerts/` |

Add validation filters in your agent code to:
- Restrict domain lookups (e.g., `.gov`, `.org`)
- Enforce schema compliance and trusted SSL certificates
- Confirm update timestamps to detect stale data

## 🧪 Example MCP Agent Query

```json
{
  "messages": [
    {
      "role": "user",
      "content": "What are the current severe weather alerts in Texas?"
    }
  ],
  "context": {
    "source": "https://api.weather.gov/alerts/active?area=TX",
    "format": "application/geo+json"
  }
}
```

### Setup
Install the required libraries

In [0]:
%pip install -U -qqq llama-index llama-index-llms-databricks mlflow python-dotenv llama-index-tools-mcp
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from llama_index.llms.databricks import Databricks
from databricks.sdk import WorkspaceClient
import mlflow

w = WorkspaceClient()

tmp_token = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = Databricks(
    model="databricks-llama-4-maverick",
    api_key=tmp_token,
    api_base=f"{w.config.host}/serving-endpoints/"
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Unexpected internal error when monkey patching `PreTrainedModel.from_pretrained`: 
PreTrainedModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.

Unexpected internal error when monkey patching `Trainer.train`: 
Trainer requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.



We will use the LlamaIndex `BasicMCPClient` to get the list of tools from the MCP server and create a `McpToolSpec` object. We can then convert this to a list of tools using the `to_tool_list_async` method.

In [0]:
%python
# Install the llama-index package
%pip install llama-index
%pip install llama-index-tools-mcp

from dotenv import load_dotenv
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec
import os

load_dotenv()

async def setup_nimble_tools():
    mcp_client = BasicMCPClient(
        "https://mcp.nimbleway.com/sse",
        headers={"Authorization": f"Bearer 1c7cdd787c154fe3a78ebcc3428d82c48f4fada38e0042c6a427fbe7e3b842ae"}
    )
    
    mcp_tool_spec = McpToolSpec(
        client=mcp_client,
        # Optional: filter to specific tools
        # allowed_tools=["nimble_web_search", "nimble_google_maps_search"]
    )
    
    tools = await mcp_tool_spec.to_tool_list_async()
    
    print(f"Loaded {len(tools)} Nimble tools:")
    for tool in tools:
        print(f"- {tool.metadata.name}: {tool.metadata.description}")
    
    return tools

# Get the tools
nimble_tools = await setup_nimble_tools()
nimble_tools

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Loaded 9 Nimble tools:
- nimble_targeted_engines: 
    Fetch lists of available Nimble Web Agent templates for targeted data extraction.

    Returns templates with: id, name, display_name, description, domain, entity_type (PDP/SERP).
    Use the 'id' field as template_number in targeted_retrieval.
    
- nimble_targeted_retrieval: 
    Executes data extraction using Nimble Web Agent's pre-trained templates.

    Leverages Nimble's intelligent extraction technology for accurate, structured data
    from e-commerce sites, job boards, and other supported domains.

    Args:
        input: Format depends on template entity_type:
            • SERP: Search query (e.g., "wireless headphones")  
            • PDP: Product identifier - Amazon: ASIN on

We will use the LlamaIndex `ReActAgent` to build our agent. We pass in the list of tools, the LLM, and a system prompt.

In [0]:
from llama_index.core.agent.workflow import ReActAgent

agent = ReActAgent(
    tools=nimble_tools,
    llm=llm,
    system_prompt="""You are a helpful research assistant with access to web scraping and data collection tools. 
    Always explain your answer in the final output. Tell the user which tools you used and how you found the information. ALWAYS Give output in one line""",
)

### Running the agent

We can run the agent with `await agent.run()`, passing in a question.


In [0]:
mlflow.llama_index.autolog()
await agent.run("what are the ongoing alerts in Texas?")

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='There are several ongoing weather alerts in Texas. According to the search results, there are River Flood Warnings in multiple counties including Hunt County, Dallas County, Rockwall County, Red River County, Navarro County, Henderson County, and Anderson County. Additionally, there are mentions of scattered storms expected tonight into Tuesday morning with a low severe weather threat, isolated large hail, and gusty winds possible. The Weather Channel reports that there are currently no weather alerts for Texas, but NBC 5 Dallas-Fort Worth lists several weather alerts including River Flood Warnings and mentions of severe thunderstorm watches and flood advisories in surrounding areas.')]), tool_calls=[ToolCallResult(tool_name='nimble_deep_web_search', tool_kwargs={'query': 'ongoing alerts in Texas', 'num_results': 5, 'search_engine': 'google_search

Trace(request_id=tr-2271dc4077244c609384cae1646c871e)

## OUTPUT

There are several ongoing weather alerts in Texas. According to the search results, there are River Flood Warnings in multiple counties including Hunt County, Dallas County, Rockwall County, Red River County, Navarro County, Henderson County, and Anderson County. Additionally, there are mentions of scattered storms expected tonight into Tuesday morning with a low severe weather threat, isolated large hail, and gusty winds possible. The Weather Channel reports that there are currently no weather alerts for Texas, but NBC 5 Dallas-Fort Worth lists several weather alerts including River Flood Warnings and mentions of severe thunderstorm watches and flood advisories in surrounding areas.